In [ ]:
import praw
import pandas as pd
import time
from datetime import datetime, timezone


In [ ]:
# Initialize Reddit instance
reddit = praw.Reddit(client_id="____",
                     client_secret="___",
                     user_agent="_____")

In [ ]:
# Search parameters
subreddit_name = "CryptoMarkets"
search_term = "melania"
start_date = datetime(2025, 1, 15, tzinfo=timezone.utc)
post_limit = 100

# Search
subreddit = reddit.subreddit(subreddit_name)
posts = subreddit.search(search_term, limit=post_limit)

# Collect comment data
all_comments = []

for idx, post in enumerate(posts, start=1):
    post_datetime = datetime.fromtimestamp(post.created_utc, tz=timezone.utc)
    if post_datetime < start_date:
        continue

    thread_title = post.title
    thread_url = f"https://www.reddit.com{post.permalink}"
    thread_time_utc = post_datetime.strftime('%Y-%m-%d %H:%M:%S')
    thread_score = post.score

    print(f"{idx}. Processing: {thread_title} | {thread_time_utc} | Score: {thread_score}")

    post.comments.replace_more(limit=None)
    for comment in post.comments.list():
        comment_time_utc = datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
        comment_text = comment.body.replace('\n', ' ').strip()
        comment_score = comment.score

        all_comments.append({
            "thread_title": thread_title,
            "thread_url": thread_url,
            "thread_time_utc": thread_time_utc,
            "thread_score": thread_score,
            "comment_time_utc": comment_time_utc,
            "comment_score": comment_score,
            "comment_text": comment_text
        })

    # Add delay in order to respect API limits
    if idx % 10 == 0:
        print("Sleeping...")
        time.sleep(3)

# Save results to CSV
df = pd.DataFrame(all_comments)
df = df.sort_values(by="comment_time_utc")
df.to_csv("subreddit_keyword_comments_filtered.csv", index=False, encoding="utf-8")

print(f"\n Saved")
